In [62]:
# -*- coding: utf_8 -*-
# import pandas as pd
from datetime import datetime
import pandas as pd
import time
import numpy as np
import re
import tensorflow as tf
from scipy import stats
import keras 
import warnings

#### 주택담보금리 

In [63]:
interest_rates = pd.read_csv('./interest_rates.csv', encoding = 'utf-8', index_col=0)
# Unnamed:0컬럼을 삭제해주는 옵션 추가 
interest_rates.rename(columns={'year_month':'transaction_year_month'},inplace=True)
interest_rates.head(10)

,household_loan,mean_loan,mortgage loan,transaction_year_month
0,5.69,5.79,5.64,200601
1,5.71,5.76,5.58,200602
2,5.68,5.89,5.46,200603
3,5.62,5.83,5.42,200604
4,5.64,5.89,5.41,200605
5,5.72,5.98,5.48,200606
6,5.95,6.10,5.79,200607
7,6.03,6.16,5.86,200608
8,5.90,6.13,5.77,200609
9,5.84,6.09,5.69,200610


#### 아파트 데이터 

In [64]:
df= pd.read_csv('./sorted_train.csv', encoding = 'utf-8',index_col=0)

C:\Python\anaconda3-52\lib\site-packages\numpy\lib\arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [65]:
df.columns
df2 = df.drop(['transaction_year_month','transaction_date','room_id','household_loan','mean_loan','floor'],axis = 1)

In [93]:
df2['parking_per_household'] = (df2.total_parking_capacity_in_site / df2.total_household_count_in_sites)
df2['transaction_year_month_date'] = pd.to_datetime(df2['transaction_year_month_date'])
# df2.parking_per_household = lambda(x : df2.total_parking_capacity_in_site, y : df2.total_household_count_in_sites, x / y )

In [91]:
# df2.tail()

# List = []
# for i in range(len(df2['transaction_year_month_date'])):
#     list1 = df2['transaction_year_month_date'][i].split('-')[0]
# List = List.append(list1)

# #     print(len(List))
# df2['year']= List
    
# # len(df2.transaction_year_month_date)

In [92]:
df2

,key,apartment_id,city,year_of_completion,exclusive_use_area,latitude,longitude,address_by_law,total_parking_capacity_in_site,total_household_count_in_sites,...,supply_area,total_household_count_of_area_type,room_count,bathroom_count,front_door_structure,transaction_real_price,mortgage loan,transaction_year_month_date,parking_per_household,year
1219584,1219594,3,seoul,1987,66.2900,37.517649,127.012646,1165010600,324.0,360,...,89.35,150,3.0,1.0,corridor,756000000,2.93,2016-04-10,0.900000,None
1474032,1474065,3,seoul,1987,66.2900,37.517649,127.012646,1165010600,324.0,360,...,89.35,150,3.0,1.0,corridor,1020000000,3.24,2017-09-30,0.900000,None
577651,577652,3,seoul,1987,84.5300,37.517649,127.012646,1165010600,324.0,360,...,101.08,210,3.0,2.0,stairway,800000000,4.80,2011-01-31,0.900000,None
435062,435062,3,seoul,1987,84.5300,37.517649,127.012646,1165010600,324.0,360,...,101.08,210,3.0,2.0,stairway,760000000,5.77,2009-09-30,0.900000,None
694846,694847,3,seoul,1987,66.2900,37.517649,127.012646,1165010600,324.0,360,...,89.35,150,3.0,1.0,corridor,570000000,4.85,2012-05-31,0.900000,None
348969,348969,3,seoul,1987,66.2900,37.517649,127.012646,1165010600,324.0,360,...,89.35,150,3.0,1.0,corridor,585000000,5.43,2009-03-20,0.900000,None
268055,268055,3,seoul,1987,84.5300,37.517649,127.012646,1165010600,324.0,360,...,101.08,210,3.0,2.0,stairway,800000000,6.70,2008-03-31,0.900000,None
349005,349005,3,seoul,1987,66.2900,37.517649,127.012646,1165010600,324.0,360,...,89.35,150,3.0,1.0,corridor,495000000,5.43,2009-03-20,0.900000,None
960913,960918,3,seoul,1987,66.2900,37.517649,127.012646,1165010600,324.0,360,...,89.35,150,3.0,1.0,corridor,660000000,3.38,2014-10-20,0.900000,None
540649,540650,3,seoul,1987,84.5300,37.517649,127.012646,1165010600,324.0,360,...,101.08,210,3.0,2.0,stairway,780000000,4.74,2010-10-31,0.900000,None


#### 구별 나누기

In [73]:
# set(df2.district)
district = pd.read_csv('./district.csv', encoding='CP949')
district.법정동코드 = district.법정동코드.astype(str)
district["법정동코드_구"] = district.법정동코드.str[2:5]
district["법정동코드_시"] = district.법정동코드.str[:2]
# district.법정동코드_구
# district.법정동코드_시
seoul_gu_code = list(district['법정동코드_구'][:25])
pusan_gu_code = list(district['법정동코드_구'][25:])
len(seoul_gu_code) # 26
len(pusan_gu_code) # 15


str

In [71]:
def extracting_gu_row():
    var1 = input('도시를 입력하세요 : ')
    var2 = input('연도를 입력하세요 : ')
    var3 = input('월을 입력하세요 : ')
    if var1 == 'seoul':
#         for i in range(len(seoul_gu_code)):
            result = df2.loc[(df2['city'] == var1) & (df2['transaction_year_month_date'].split('-')[0] == var2) & (df2['transaction_year_month_date'].split('-')[1] == var3)]
    else:
#         for j in range(len(pusan_gu_code)):
            result = df2.loc[(df2['city'] == var1) & (df2['transaction_year_month_date'].split('-')[0] == var2) & (df2['transaction_year_month_date'].split('-')[1] == var3)]
    
    return result

# df2.loc[(df2.city == 'seoul') & (df2.year == '2006') & (df2.district == '650') & (df2.month == '01'), : ]

extracting_gu_row()

도시를 입력하세요 : seoul
연도를 입력하세요 : 2006
월을 입력하세요 : 01


AttributeError: 'Series' object has no attribute 'split'

In [ ]:
df2

In [94]:

# get 'district' and 'dong' feature
# df2['address_by_law'] = df2['address_by_law'].astype(str)
# df2['district'] = df2.address_by_law.apply(lambda x: x[2:5])
# df2['dong'] = df2.address_by_law.apply(lambda x: x[5:-2])
# df2['year'] = df2.transaction_year_month_date.apply(lambda x: x[:4])
# df2['month'] = df2.transaction_year_month_date.apply(lambda x : x[5:7])

# df2.loc[(df2.year == '2006') & (df2.district == '650') & (df2.month == '01'), : ]
districts=list(set(df2.district))
k=list(set(df2.year)) 
years = ['2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018'] 
months = ['01','02','03','04','05','06','07','08','09','10','11','12'] 
# for i in range(len(years)): 
#     for j in range(len(months)): 
#         for p in range(len(districts)): 
#         df2.loc[(df2.year == years[i]) & (df2.month == months[j]) & (df2.district == districts[p]), :]




,key,apartment_id,city,year_of_completion,exclusive_use_area,latitude,longitude,address_by_law,total_parking_capacity_in_site,total_household_count_in_sites,...,bathroom_count,front_door_structure,transaction_real_price,mortgage loan,transaction_year_month_date,parking_per_household,district,dong,year,month
1219584,1219594,3,seoul,1987,66.2900,37.517649,127.012646,1165010600,324.0,360,...,1.0,corridor,756000000,2.93,2016-04-10,0.900000,650,106,2016,04
1474032,1474065,3,seoul,1987,66.2900,37.517649,127.012646,1165010600,324.0,360,...,1.0,corridor,1020000000,3.24,2017-09-30,0.900000,650,106,2017,09
577651,577652,3,seoul,1987,84.5300,37.517649,127.012646,1165010600,324.0,360,...,2.0,stairway,800000000,4.80,2011-01-31,0.900000,650,106,2011,01
435062,435062,3,seoul,1987,84.5300,37.517649,127.012646,1165010600,324.0,360,...,2.0,stairway,760000000,5.77,2009-09-30,0.900000,650,106,2009,09
694846,694847,3,seoul,1987,66.2900,37.517649,127.012646,1165010600,324.0,360,...,1.0,corridor,570000000,4.85,2012-05-31,0.900000,650,106,2012,05
348969,348969,3,seoul,1987,66.2900,37.517649,127.012646,1165010600,324.0,360,...,1.0,corridor,585000000,5.43,2009-03-20,0.900000,650,106,2009,03
268055,268055,3,seoul,1987,84.5300,37.517649,127.012646,1165010600,324.0,360,...,2.0,stairway,800000000,6.70,2008-03-31,0.900000,650,106,2008,03
349005,349005,3,seoul,1987,66.2900,37.517649,127.012646,1165010600,324.0,360,...,1.0,corridor,495000000,5.43,2009-03-20,0.900000,650,106,2009,03
960913,960918,3,seoul,1987,66.2900,37.517649,127.012646,1165010600,324.0,360,...,1.0,corridor,660000000,3.38,2014-10-20,0.900000,650,106,2014,10
540649,540650,3,seoul,1987,84.5300,37.517649,127.012646,1165010600,324.0,360,...,2.0,stairway,780000000,4.74,2010-10-31,0.900000,650,106,2010,10


In [76]:
# parking_per_household
# df2.parking_per_household=df2.parking_per_household.fillna(value=df2['parking_per_household'].mean())

# heat_fuel 
# df2.heat_fuel=df2.heat_fuel.fillna(value='Not Certified')
# df2.heat_fuel.value_counts()

# heat_type
# df2.heat_type=df2.heat_type.fillna(value='Not Certified')
# df2.heat_type.isna().sum()

# front_door_structure
# df2.front_door_structure = df2.front_door_structure.fillna(method = 'bfill')
# df2.front_door_structure.value_counts()
# df2.front_door_structure=df2.front_door_structure.replace('-','Not Certified')
# df2.front_door_structure = df2.front_door_structure.fillna(value='Not Certified')



df2.isna().sum()


# df_train.heat_fuel = df_train.heat_fuel.replace('-', 'Not Certified')
# df2.parking_per_household.isna().sum()

# df_train.heat_fuel.isna().sum()
# df_train.front_door_structure.value_counts()
# df_train.total_parking_capacity_in_site.value_counts()
# sorted_train=df_train.sort_values('apartment_id')
# df_train.front_door_structure.replace('-',0)


key                                       0
apartment_id                              0
city                                      0
year_of_completion                        0
exclusive_use_area                        0
latitude                                  0
longitude                                 0
address_by_law                            0
total_parking_capacity_in_site        91813
total_household_count_in_sites            0
apartment_building_count_in_sites         0
tallest_building_in_sites                 9
lowest_building_in_sites                  9
heat_type                                 0
heat_fuel                                 0
supply_area                               0
total_household_count_of_area_type        0
room_count                              691
bathroom_count                          691
front_door_structure                      0
transaction_real_price                    0
mortgage loan                             0
transaction_year_month_date     

In [ ]:
# room_count
from sklearn.linear_model import LinearRegression 
lr = LinearRegression()
X,y = df2

In [74]:
sorted_train.to_csv('./sorted_train.csv', encoding = 'utf-8', sep = ',')

NameError: name 'sorted_train' is not defined

#### 결측값 조정(평균값 채워넣기, 앞방향으로 채우기)

In [7]:
# df_train.isna().sum()
# 결측값 리스트 
'''
total_parking_capacity_in_site        91813
heat_type                              2017
heat_fuel                              9667
room_count                              691
bathroom_count                          691
front_door_structure                  13892
'''

# 주차대수는 컬럼의 평균으로 
df_train.total_parking_capacity_in_site=df_train.total_parking_capacity_in_site.fillna(df_train.total_parking_capacity_in_site.mean())
# 난방 종류는 ffill
df_train.heat_type = df_train.heat_type.fillna(method='ffill')
# 난방 연료는 ffill
df_train.heat_fuel = df_train.heat_fuel.fillna(method='ffill')
# room_count는 컬럼평균
df_train.room_count = df_train.room_count.fillna(df_train.room_count.mean())
# bathromm_count도 컬럼평균
df_train.bathroom_count = df_train.bathroom_count.fillna(df_train.bathroom_count.mean())
# front_door_structure는 ffill 
df_train.front_door_structure = df_train.front_door_structure.fillna(method='ffill')

In [8]:
# df_train.isna().sum()
df_train.head()
df_train.room_id

0           91120
1           91119
2            8430
3            5839
4            5836
5            5831
6            5833
7           11862
8            5843
9            5842
10           5851
11           5847
12          11866
13          11867
14           5843
15           5848
16          11868
17           5841
18           5845
19          11864
20          11865
21           5842
22           7222
23         146919
24           3965
25           3966
26           3965
27           3966
28         146919
29           3965
            ...  
1601428     44385
1601429     10537
1601430     17020
1601431      5871
1601432      5870
1601433     13062
1601434     48522
1601435     54538
1601436     56441
1601437     56442
1601438     57266
1601439     57267
1601440     57268
1601441     57269
1601442     26219
1601443     95331
1601444     10538
1601445     37397
1601446    147606
1601447     54538
1601448     56043
1601449     56044
1601450     92198
1601451     34262
1601452   

In [9]:
df_train.columns

'''
Lat long coordinates have a problem that they are 2 features 
that represent a three dimensional space. 
This means that the long coordinate goes all around, 
which means the two most extreme values are actually very close together.
I've dealt with this problem a few times and
what I do in this case is map them to x, y and z coordinates. 
This means close points in these 3 dimensions are also close in reality. 
Depending on the use case you can disregard the changes in height 
and map them to a perfect sphere. 
These features can then be standardized properly. 

x = np.cos(lat) * np.cos(lon)
y = np.cos(lat) * np.sin(lon), 
z = np.sin(lat) 
'''
df_train2 = df_train.copy()
df_train3 = df_train2.drop(['transaction_year_month', 'transaction_date',
                           'latitude','longitude','address_by_law','room_id','household_loan','mean_loan'],axis=1)
# df_train3 = pd.DataFrame(df_train3, columns=)

### 범주형 변수, one-hot encoding 하기

In [10]:
df_train3.head()
df_train4 = pd.get_dummies(df_train3)
df_train4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1601458 entries, 0 to 1601457
Data columns (total 29 columns):
key                                   1601458 non-null int64
apartment_id                          1601458 non-null int64
year_of_completion                    1601458 non-null int64
exclusive_use_area                    1601458 non-null float64
floor                                 1601458 non-null int64
total_parking_capacity_in_site        1601458 non-null float64
total_household_count_in_sites        1601458 non-null int64
apartment_building_count_in_sites     1601458 non-null int64
tallest_building_in_sites             1601449 non-null float64
lowest_building_in_sites              1601449 non-null float64
supply_area                           1601458 non-null float64
total_household_count_of_area_type    1601458 non-null int64
room_count                            1601458 non-null float64
bathroom_count                        1601458 non-null float64
transaction_real_pr

In [11]:
# 임의의 열을 임의의 위치로 이동 
'''
df10 = pd.DataFrame({"A": [1,2,3], 
                   "B": [2,4,8], 
                   "C": [5,5,5]})

cols = df10.columns.tolist()
column_to_move = "A"
new_position = -1

cols.insert(new_position, cols.pop(cols.index(column_to_move)))
df10 = df10[cols]
'''
cols = df_train4.columns.tolist()
# len(cols) # 전체 29 
column_to_move = "transaction_real_price"
new_position = 29 

cols.insert(new_position,cols.pop(cols.index(column_to_move)))
df_train5 = df_train4[cols] 
df_train5 # 실거래가격이 제일 뒤쪽으로 이동 

,key,apartment_id,year_of_completion,exclusive_use_area,floor,total_parking_capacity_in_site,total_household_count_in_sites,apartment_building_count_in_sites,tallest_building_in_sites,lowest_building_in_sites,...,heat_type_district,heat_type_individual,heat_fuel_-,heat_fuel_cogeneration,heat_fuel_gas,front_door_structure_-,front_door_structure_corridor,front_door_structure_mixed,front_door_structure_stairway,transaction_real_price
0,0,5584,1999,47.4300,6,163.000000,136,1,8.0,4.0,...,0,1,0,0,1,0,1,0,0,215000000
1,1,5584,1999,44.3700,8,163.000000,136,1,8.0,4.0,...,0,1,0,0,1,0,1,0,0,200000000
2,2,5059,1992,54.7000,8,902.000000,585,5,14.0,9.0,...,0,1,0,0,1,0,1,0,0,168000000
3,3,2816,1993,64.6600,11,902.000000,919,7,15.0,11.0,...,0,1,0,0,1,0,1,0,0,165000000
4,4,2816,1993,106.6200,7,902.000000,919,7,15.0,11.0,...,0,1,0,0,1,0,0,0,1,280000000
5,5,2815,2000,84.9200,9,1365.000000,964,12,23.0,10.0,...,0,1,0,0,1,0,0,0,1,415000000
6,6,2815,2000,60.0000,13,1365.000000,964,12,23.0,10.0,...,0,1,0,0,1,0,1,0,0,267000000
7,7,9867,2005,84.7600,10,652.000000,461,9,23.0,6.0,...,0,1,0,0,1,0,0,0,1,415000000
8,8,2818,1999,84.8800,18,2091.000000,2282,19,20.0,8.0,...,0,1,0,0,1,0,0,0,1,310000000
9,9,2817,2002,59.9400,12,4329.000000,5150,42,18.0,11.0,...,0,1,0,0,1,0,1,0,0,319000000


### data regularization

In [97]:
# df_train['transaction_year_month_date']=pd.to_datetime(df_train['transaction_year_month_date']) 
from datetime import datetime
import time 
Timestamp = []
for i in range(len(df_train5.transaction_year_month_date)):
    timestamp = time.mktime(df_train5.transaction_year_month_date[i].timetuple())
    Timestamp.append(timestamp) 

In [98]:
t1=Timestamp
t1_d = pd.DataFrame(t1)

In [100]:
df_train5["timestamp"]=t1
# df_train.sort_values('timestamp')
# timestamp가 날짜별로 움직이므로, 그대로 log화 해도 될 것이다. 
# 여기까지 완료.
df_train5

C:\Python\anaconda3-52\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,key,apartment_id,year_of_completion,exclusive_use_area,floor,total_parking_capacity_in_site,total_household_count_in_sites,apartment_building_count_in_sites,tallest_building_in_sites,lowest_building_in_sites,...,heat_type_individual,heat_fuel_-,heat_fuel_cogeneration,heat_fuel_gas,front_door_structure_-,front_door_structure_corridor,front_door_structure_mixed,front_door_structure_stairway,transaction_real_price,timestamp
0,0,5584,1999,47.4300,6,163.000000,136,1,8.0,4.0,...,1,0,0,1,0,1,0,0,215000000,1.137683e+09
1,1,5584,1999,44.3700,8,163.000000,136,1,8.0,4.0,...,1,0,0,1,0,1,0,0,200000000,1.137683e+09
2,2,5059,1992,54.7000,8,902.000000,585,5,14.0,9.0,...,1,0,0,1,0,1,0,0,168000000,1.137683e+09
3,3,2816,1993,64.6600,11,902.000000,919,7,15.0,11.0,...,1,0,0,1,0,1,0,0,165000000,1.137683e+09
4,4,2816,1993,106.6200,7,902.000000,919,7,15.0,11.0,...,1,0,0,1,0,0,0,1,280000000,1.137683e+09
5,5,2815,2000,84.9200,9,1365.000000,964,12,23.0,10.0,...,1,0,0,1,0,0,0,1,415000000,1.137683e+09
6,6,2815,2000,60.0000,13,1365.000000,964,12,23.0,10.0,...,1,0,0,1,0,1,0,0,267000000,1.137683e+09
7,7,9867,2005,84.7600,10,652.000000,461,9,23.0,6.0,...,1,0,0,1,0,0,0,1,415000000,1.137683e+09
8,8,2818,1999,84.8800,18,2091.000000,2282,19,20.0,8.0,...,1,0,0,1,0,0,0,1,310000000,1.137683e+09
9,9,2817,2002,59.9400,12,4329.000000,5150,42,18.0,11.0,...,1,0,0,1,0,1,0,0,319000000,1.137683e+09


In [96]:
from sklearn.preprocessing import StandardScaler
stdscaler = StandardScaler()
x_dat_std = stdscaler.fit_transform(df_train5)

TypeError: float() argument must be a string or a number, not 'Timestamp'